In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from secret import *
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(
    url = "bolt://localhost:7687",
    username = "neo4j",
    password = "health1234"
)

In [3]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Disease', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'Symptom', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        [{'properties': [{'property': 'prob', 'type': 'FLOAT'}], 'type': 'HAS'}]
        The relationships are the following:
        ['(:Disease)-[:HAS]->(:Symptom)']
        


In [7]:
import pandas as pd
df = pd.read_csv('health-kg.csv')

In [16]:
REFLACX_LESION_LABEL_COLS = [
    "Fibrosis",
    "Quality issue",
    "Wide mediastinum",
    "Fracture",
    "Airway wall thickening",

    ######################
    "Hiatal hernia",
    "Acute fracture",
    "Interstitial lung disease",
    "Enlarged hilum",
    "Abnormal mediastinal contour",
    "High lung volume / emphysema",
    "Pneumothorax",
    "Lung nodule or mass",
    "Groundglass opacity",
    ######################
    "Pulmonary edema",
    "Enlarged cardiac silhouette",
    "Consolidation",
    "Atelectasis",
    "Pleural abnormality",
    # "Support devices",
]

In [15]:
"atelectasis" in [d.lower() for d in list(df.Diseases)]

False

In [22]:
contain_l = []
for l in REFLACX_LESION_LABEL_COLS:
    if l.lower() in [d.lower() for d in list(df.Diseases)]:
        contain_l.append(l)
print(contain_l)

[]


In [24]:
CHEXPERT_LABEL_COLS = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Enlarged Cardiomediastinum",
    "Fracture",
    "Lung Lesion",
    "Lung Opacity",
    "No Finding",
    "Pleural Effusion",
    "Pleural Other",
    "Pneumonia",
    "Pneumothorax",
    "Support Devices",
]

In [25]:
contain_l = []
for l in CHEXPERT_LABEL_COLS:
    if l.lower() in [d.lower() for d in list(df.Diseases)]:
        contain_l.append(l)
print(contain_l)

['Pleural Effusion', 'Pneumonia', 'Pneumothorax']


In [4]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), graph=graph, verbose=True
)

In [26]:
chain.run("What is pleural effusion?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: 'pleural effusion'})-[:HAS]->(s:Symptom)
RETURN d, s
Full Context:
[{'d': {'name': 'pleural effusion'}, 's': {'name': 'low oxygen in the body'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'wheezing'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'infection'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'fluid in the abdomen'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'cough with phlegm'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'fatigue'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'lethargy'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'fluid in the lungs'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'dry cough'}}, {'d': {'name': 'pleural effusion'}, 's': {'name': 'fever'}}]

> Finished chain.


'Pleural effusion refers to the buildup of fluid in the space between the layers of tissue that line the lungs and chest cavity. It can cause symptoms such as low oxygen in the body, wheezing, infection, fluid in the abdomen, cough with phlegm, fatigue, lethargy, fluid in the lungs, dry cough, and fever.'

In [18]:
chain.run("What are the symptoms of pleural effusion?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: 'pleural effusion'})-[:HAS]->(s:Symptom)
RETURN s.name
Full Context:
[{'s.name': 'low oxygen in the body'}, {'s.name': 'wheezing'}, {'s.name': 'infection'}, {'s.name': 'fluid in the abdomen'}, {'s.name': 'cough with phlegm'}, {'s.name': 'fatigue'}, {'s.name': 'lethargy'}, {'s.name': 'fluid in the lungs'}, {'s.name': 'dry cough'}, {'s.name': 'fever'}]

> Finished chain.


'The symptoms of pleural effusion may include low oxygen in the body, wheezing, infection, fluid in the abdomen, cough with phlegm, fatigue, lethargy, fluid in the lungs, dry cough, and fever.'

In [27]:
chain.run("What are the causes of pleural effusion?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: 'pleural effusion'})-[:HAS]->(s:Symptom)
RETURN s.name
Full Context:
[{'s.name': 'low oxygen in the body'}, {'s.name': 'wheezing'}, {'s.name': 'infection'}, {'s.name': 'fluid in the abdomen'}, {'s.name': 'cough with phlegm'}, {'s.name': 'fatigue'}, {'s.name': 'lethargy'}, {'s.name': 'fluid in the lungs'}, {'s.name': 'dry cough'}, {'s.name': 'fever'}]

> Finished chain.


'The causes of pleural effusion can include low oxygen in the body, wheezing, infection, fluid in the abdomen, cough with phlegm, fatigue, lethargy, fluid in the lungs, dry cough, and fever.'

In [37]:
documents = text_splitter.create_documents([raw_documents[0].page_content])

In [25]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [38]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [39]:
graph = index_creator.from_text(documents[0].page_content)

In [28]:
graph

In [43]:
graph.draw_graphviz()

[('Atelectasis', 'a lung', 'is the collapse or closure of'),
 ('Atelectasis', 'reduced or absent gas exchange', 'results in'),
 ('Atelectasis', 'unilateral', 'is usually'),
 ('Atelectasis', 'part or all of one lung', 'affects'),
 ('Atelectasis', 'the alveoli are deflated down', 'is a condition where')]

In [13]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [33]:
from langchain. graphs.neo4j_graph import Neo4jGraph

In [44]:
from IPython.display import SVG
graph.draw_graphviz(layout="dot", filename="web.svg")
SVG('web.svg')

ImportError: Could not import pygraphviz python package. Please install it with:`pip install pygraphviz`.

In [14]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [15]:
neo4j_vector = Neo4jVector.from_documents(
    documents,
    # hf,
    OpenAIEmbeddings(),
    url=NOE4J_URL,
    username="neo4j",
    password=NOE4J_PASSWORD,
)

In [16]:
query = "What is atelectasis?"
vector_results = neo4j_vector.similarity_search(query, k=2)
for i, res in enumerate(vector_results):
    print(res.page_content)
    if i != len(vector_results)-1:
        print()
vector_result = vector_results[0].page_content

Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange.  It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down

Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange.  It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [18]:
graph = Neo4jGraph(
        url = NOE4J_URL,
    username = 'neo4j',
    password = NOE4J_PASSWORD
)

print(graph.schema)


        Node properties are the following:
        [{'labels': 'Chunk', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'text', 'type': 'STRING'}, {'property': 'embedding', 'type': 'LIST'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [19]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

graph_result = chain.run("What are the symptoms of Atelectasis")
graph_result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE c.text = 'Atelectasis'
RETURN s.property
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question. Atelectasis is a medical condition characterized by the collapse or closure of a lung or a part of it, resulting in reduced or absent gas exchange. The symptoms of atelectasis can vary depending on the severity and location of the collapse. Common symptoms may include shortness of breath, rapid breathing, chest pain, coughing, and low oxygen levels. However, it is always best to consult a healthcare professional for an accurate diagnosis and appropriate treatment."